In [16]:
import pandas as pd
import networkx as nx
import numpy as np
from tqdm import tqdm

In [2]:
df_distance_km = "../dataset/df_distance_km.xlsx"
df = pd.read_excel(df_distance_km)
df.head()

,CLIENTES,Cliente_1,Cliente_2,Cliente_3,Cliente_4,Cliente_5,Cliente_6,Cliente_7,Cliente_8,Cliente_9,...,Cliente_12,Cliente_13,Cliente_14,Cliente_15,Cliente_16,Cliente_17,Cliente_18,Cliente_19,Cliente_20,Almacén
0,Cliente_1,0.0000,7.5625,15.5365,1.1998,4.7145,1.7407,7.9408,17.1947,4.2933,...,6.0225,5.4470,2.2133,11.1505,1.5775,10.8288,9.1456,20.4871,22.1445,3.6114
1,Cliente_2,7.5625,0.0000,3.3838,7.7433,14.5720,8.5237,0.4847,13.7974,10.1522,...,10.1049,2.6961,13.4907,18.0835,7.0275,19.8218,8.2737,9.6369,19.1038,10.7361
2,Cliente_3,15.5365,3.3838,0.0000,12.5438,0.0000,0.0000,0.0000,16.0355,13.9120,...,12.3430,5.0114,15.7289,17.9217,9.6824,22.0599,10.5118,7.7574,16.5997,13.9021
3,Cliente_4,1.1998,7.7433,12.5438,0.0000,5.0721,0.9119,7.5798,17.4095,3.5781,...,6.2330,4.7117,2.8799,11.3610,1.3127,11.1926,9.3561,20.7019,21.1518,3.3673
4,Cliente_5,4.7145,14.5720,0.0000,5.0721,0.0000,4.8187,0.0000,0.0000,0.0000,...,6.8738,9.1064,3.6476,12.0019,5.4347,5.2872,9.9969,21.3384,25.5947,4.5417


In [3]:
df_clientes = df["CLIENTES"]
df_clientes

0      Cliente_1
1      Cliente_2
2      Cliente_3
3      Cliente_4
4      Cliente_5
5      Cliente_6
6      Cliente_7
7      Cliente_8
8      Cliente_9
9     Cliente_10
10    Cliente_11
11    Cliente_12
12    Cliente_13
13    Cliente_14
14    Cliente_15
15    Cliente_16
16    Cliente_17
17    Cliente_18
18    Cliente_19
19    Cliente_20
20       Almacén
Name: CLIENTES, dtype: object

In [4]:
df = df.drop(columns=["CLIENTES"])
distance_km_array = df.to_numpy()
distance_km_array

array([[ 0.    ,  7.5625, 15.5365,  1.1998,  4.7145,  1.7407,  7.9408,
        17.1947,  4.2933,  3.2659,  2.1866,  6.0225,  5.447 ,  2.2133,
        11.1505,  1.5775, 10.8288,  9.1456, 20.4871, 22.1445,  3.6114],
       [ 7.5625,  0.    ,  3.3838,  7.7433, 14.572 ,  8.5237,  0.4847,
        13.7974, 10.1522,  7.1521, 14.8113, 10.1049,  2.6961, 13.4907,
        18.0835,  7.0275, 19.8218,  8.2737,  9.6369, 19.1038, 10.7361],
       [15.5365,  3.3838,  0.    , 12.5438,  0.    ,  0.    ,  0.    ,
        16.0355, 13.912 , 13.0649, 17.0494, 12.343 ,  5.0114, 15.7289,
        17.9217,  9.6824, 22.0599, 10.5118,  7.7574, 16.5997, 13.9021],
       [ 1.1998,  7.7433, 12.5438,  0.    ,  5.0721,  0.9119,  7.5798,
        17.4095,  3.5781,  3.3451,  2.8532,  6.233 ,  4.7117,  2.8799,
        11.361 ,  1.3127, 11.1926,  9.3561, 20.7019, 21.1518,  3.3673],
       [ 4.7145, 14.572 ,  0.    ,  5.0721,  0.    ,  4.8187,  0.    ,
         0.    ,  0.    ,  7.217 ,  2.6253,  6.8738,  9.1064,  3.6476,
  

In [5]:
import numpy as np
from deap import base, creator, tools, algorithms
import random

In [6]:
# Índice del último cliente (inicio y fin)
ultimo_cliente = len(distance_km_array) - 1

# 2. Configurar DEAP
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizar distancia
creator.create("Individual", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

# Función para crear un individuo
def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente
    random.shuffle(puntos)  # Mezcla los puntos
    return [ultimo_cliente] + puntos + [ultimo_cliente]  # Ruta completa

# Registrar funciones en DEAP
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Función para evaluar la distancia total
def evaluate(individual):
    total_distance = 0
    for i in range(len(individual) - 1):
        city_a = individual[i]
        city_b = individual[i + 1]
        total_distance += distance_km_array[city_a][city_b]
    return (total_distance,)

toolbox.register("evaluate", evaluate)

# Función para cruzar dos individuos respetando extremos
def crossover_individuals(ind1, ind2):
    ruta1 = ind1[1:-1]  # Intermedios de ind1
    ruta2 = ind2[1:-1]  # Intermedios de ind2

    size = len(ruta1)
    p1, p2 = sorted(random.sample(range(size), 2))  # Dos puntos de cruce

    child1, child2 = ruta1[:], ruta2[:]

    # Intercambiar segmentos
    child1[p1:p2], child2[p1:p2] = ruta2[p1:p2], ruta1[p1:p2]

    # Recombinar las rutas
    return [ind1[0]] + child1 + [ind1[-1]], [ind2[0]] + child2 + [ind2[-1]]

toolbox.register("mate", crossover_individuals)

# Función para mutar un individuo respetando extremos
def mutate_individual(individual):
    ruta_intermedia = individual[1:-1]  # Excluye el primer y último punto
    random.shuffle(ruta_intermedia)  # Mezcla los puntos intermedios
    return [individual[0]] + ruta_intermedia + [individual[-1]],  # Recompone la ruta

toolbox.register("mutate", mutate_individual)

# Selección por torneo
toolbox.register("select", tools.selTournament, tournsize=3)

# 3. Configuración del algoritmo genético
def main():
    random.seed(42)  # Para reproducibilidad
    
    # Crear población inicial
    population = toolbox.population(n=100)
    
    # Parámetros del algoritmo
    ngen = 200  # Número de generaciones
    cxpb = 0.7  # Probabilidad de cruce
    mutpb = 0.2  # Probabilidad de mutación
    
    # Estadísticas
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("min", min)
    stats.register("avg", lambda x: sum(x) / len(x))
    
    # Algoritmo genético
    for gen in range(ngen):
        # Selección
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        # Cruzamiento
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        # Mutación
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        # Evaluar individuos sin fitness
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        # Reemplazar la población
        population[:] = offspring
        
        # Mostrar estadísticas
        fits = [ind.fitness.values[0] for ind in population]
        print(f"Generación {gen + 1}: Min = {min(fits)}, Avg = {sum(fits) / len(fits)}")
    
    # Mejor individuo encontrado
    best_ind = tools.selBest(population, 1)[0]
    print("\nMejor ruta encontrada (índices):", best_ind)
    
    print("Mejor ruta encontrada (clientes):", [f"Cliente_{i+1}" for i in best_ind])
    print("Distancia total:", evaluate(best_ind)[0])

if __name__ == "__main__":
    main()

Generación 1: Min = 166.2326, Avg = 221.21610600000002
Generación 2: Min = 166.2326, Avg = 204.7791670000001
Generación 3: Min = 166.2326, Avg = 194.31107600000013
Generación 4: Min = 166.2326, Avg = 186.86794600000002
Generación 5: Min = 166.2326, Avg = 179.50194700000003
Generación 6: Min = 166.2326, Avg = 171.0173989999999
Generación 7: Min = 166.2326, Avg = 166.58987399999975
Generación 8: Min = 166.2326, Avg = 166.23259999999974
Generación 9: Min = 166.2326, Avg = 166.23259999999974
Generación 10: Min = 166.2326, Avg = 166.23259999999974
Generación 11: Min = 166.2326, Avg = 166.23259999999974
Generación 12: Min = 166.2326, Avg = 166.23259999999974
Generación 13: Min = 166.2326, Avg = 166.23259999999974
Generación 14: Min = 166.2326, Avg = 166.23259999999974
Generación 15: Min = 166.2326, Avg = 166.23259999999974
Generación 16: Min = 166.2326, Avg = 166.23259999999974
Generación 17: Min = 166.2326, Avg = 166.23259999999974
Generación 18: Min = 166.2326, Avg = 166.23259999999974
Gen

In [7]:
df_vehiculos = pd.read_excel("../dataset/df_vehicle.xlsx")
df_vehiculos.head()

,vehiculo_id,capacidad_kg,costo_km,autonomia_km
0,1,2026,0.20,603
1,2,4362,0.14,630
2,3,4881,0.20,664
3,4,3321,0.19,514
4,5,10000,0.32,350


In [9]:
df_time = pd.read_excel("../dataset/df_distance_min.xlsx")

#df_time= df_time.drop(columns=["CLIENTES"])
df_time_array = df.to_numpy()
df_time_array

array([[ 0.    ,  7.5625, 15.5365,  1.1998,  4.7145,  1.7407,  7.9408,
        17.1947,  4.2933,  3.2659,  2.1866,  6.0225,  5.447 ,  2.2133,
        11.1505,  1.5775, 10.8288,  9.1456, 20.4871, 22.1445,  3.6114],
       [ 7.5625,  0.    ,  3.3838,  7.7433, 14.572 ,  8.5237,  0.4847,
        13.7974, 10.1522,  7.1521, 14.8113, 10.1049,  2.6961, 13.4907,
        18.0835,  7.0275, 19.8218,  8.2737,  9.6369, 19.1038, 10.7361],
       [15.5365,  3.3838,  0.    , 12.5438,  0.    ,  0.    ,  0.    ,
        16.0355, 13.912 , 13.0649, 17.0494, 12.343 ,  5.0114, 15.7289,
        17.9217,  9.6824, 22.0599, 10.5118,  7.7574, 16.5997, 13.9021],
       [ 1.1998,  7.7433, 12.5438,  0.    ,  5.0721,  0.9119,  7.5798,
        17.4095,  3.5781,  3.3451,  2.8532,  6.233 ,  4.7117,  2.8799,
        11.361 ,  1.3127, 11.1926,  9.3561, 20.7019, 21.1518,  3.3673],
       [ 4.7145, 14.572 ,  0.    ,  5.0721,  0.    ,  4.8187,  0.    ,
         0.    ,  0.    ,  7.217 ,  2.6253,  6.8738,  9.1064,  3.6476,
  

In [12]:
df_pedidos = pd.read_excel("../dataset/df_orders.xlsx")
df_pedidos.head()

,cliente,mes_anio,order_demand
0,Cliente_1,12-2024,909
1,Cliente_2,12-2024,959
2,Cliente_3,12-2024,960
3,Cliente_4,12-2024,980
4,Cliente_5,12-2024,979


In [17]:
# Crear un individuo con rutas para todos los vehículos
def create_individual():
    puntos = list(range(len(distance_km_array) - 1))  # Excluye el último cliente
    random.shuffle(puntos)  # Mezcla los puntos
    rutas = [[] for _ in range(len(df_vehiculos))]  # Una ruta por vehículo
    pedidos_asignados = {i: 0 for i in range(len(df_vehiculos))}  # Capacidad usada por cada vehículo
    
    for punto in puntos:
        pedido_size = df_pedidos.loc[punto, 'order_demand']  # Tamaño del pedido
        # Buscar un vehículo disponible
        for i, capacidad_usada in pedidos_asignados.items():
            if capacidad_usada + pedido_size <= df_vehiculos.loc[i, 'capacidad_kg']:
                rutas[i].append(punto)
                pedidos_asignados[i] += pedido_size
                break
    
    # Agregar inicio y fin para cada ruta
    for ruta in rutas:
        ruta.insert(0, ultimo_cliente)  # Inicio
        ruta.append(ultimo_cliente)  # Fin
    
    return rutas

# Función para evaluar un individuo
def evaluate(individual):
    total_distance = 0
    total_time = 0
    penalty = 0  # Penalización por restricciones

    for i, ruta in enumerate(individual):
        ruta_distance = 0
        ruta_time = 0
        carga_total = 0
        
        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]
            
            distance = distance_km_array[city_a][city_b]
            time = df_time_array[city_a][city_b]
            pedido_size = df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
            
            # Verificar si los nodos están conectados
            if distance == 0:  # Nodo no conectado
                penalty += 1000  # Penalización alta por conexión inválida
                continue
            
            ruta_distance += distance
            ruta_time += time
            carga_total += pedido_size
        
        # Verificar restricciones
        if ruta_distance > df_vehiculos.loc[i, 'autonomia_km']:
            penalty += (ruta_distance - df_vehiculos.loc[i, 'autonomia_km']) * 10
        if carga_total > df_vehiculos.loc[i, 'capacidad_kg']:
            penalty += (carga_total - df_vehiculos.loc[i, 'capacidad_kg']) * 10
        
        total_distance += ruta_distance
        total_time += ruta_time

    # Retornar distancia total (con penalización) y tiempo total
    return (total_distance + penalty, total_time)

# Ajustes adicionales a main y presentación de resultados:
def main():
    random.seed(42)
    
    # Crear población inicial
    population = toolbox.population(n=100)
    
    ngen = 200  # Número de generaciones
    cxpb = 0.7  # Probabilidad de cruce
    mutpb = 0.2  # Probabilidad de mutación
    
    for gen in tqdm(range(ngen)):
        offspring = toolbox.select(population, len(population))
        offspring = list(map(toolbox.clone, offspring))
        
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < cxpb:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
        
        for mutant in offspring:
            if random.random() < mutpb:
                toolbox.mutate(mutant)
                del mutant.fitness.values
        
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = map(toolbox.evaluate, invalid_ind)
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit
        
        population[:] = offspring
        
        fits = [ind.fitness.values for ind in population]
        min_dist = min(f[0] for f in fits)
        min_time = min(f[1] for f in fits)
        #print(f"Generación {gen + 1}: Distancia Min = {min_dist}, Tiempo Min = {min_time}")

    best_ind = tools.selBest(population, 1)[0]
    print("\nMejor solución encontrada:")

    for i, ruta in enumerate(best_ind):
        ruta_distance = 0
        ruta_time = 0
        carga_total = 0
        
        for j in range(len(ruta) - 1):
            city_a = ruta[j]
            city_b = ruta[j + 1]
            ruta_distance += distance_km_array[city_a][city_b]
            ruta_time += df_time_array[city_a][city_b]
            carga_total += df_pedidos.loc[city_b, 'order_demand'] if city_b != ultimo_cliente else 0
        
        print(f"Vehículo {i + 1}:")
        print(f"  Ruta: {ruta}")
        print(f"  Distancia total: {ruta_distance:.2f} km (Máxima permitida: {df_vehiculos.loc[i, 'autonomia_km']} km)")
        print(f"  Tiempo total: {ruta_time:.2f} minutos")
        print(f"  Carga total: {carga_total:.2f} (Máxima permitida: {df_vehiculos.loc[i, 'capacidad_kg']})")

if __name__ == "__main__":
    main()


100%|██████████| 200/200 [00:01<00:00, 148.09it/s]


Mejor solución encontrada:
Vehículo 1:
  Ruta: [20, 5, 15, 7, 1, 20]
  Distancia total: 43.89 km (Máxima permitida: 603 km)
  Tiempo total: 43.89 minutos
  Carga total: 3678.00 (Máxima permitida: 2026)
Vehículo 2:
  Ruta: [20, 19, 6, 18, 11, 20]
  Distancia total: 68.54 km (Máxima permitida: 630 km)
  Tiempo total: 68.54 minutos
  Carga total: 3680.00 (Máxima permitida: 4362)
Vehículo 3:
  Ruta: [20, 3, 10, 4, 20]
  Distancia total: 13.39 km (Máxima permitida: 664 km)
  Tiempo total: 13.39 minutos
  Carga total: 2901.00 (Máxima permitida: 4881)
Vehículo 4:
  Ruta: [20, 8, 9, 16, 20]
  Distancia total: 27.33 km (Máxima permitida: 514 km)
  Tiempo total: 27.33 minutos
  Carga total: 2818.00 (Máxima permitida: 3321)
Vehículo 5:
  Ruta: [20, 0, 14, 12, 20]
  Distancia total: 32.82 km (Máxima permitida: 350 km)
  Tiempo total: 32.82 minutos
  Carga total: 2797.00 (Máxima permitida: 10000)
Vehículo 6:
  Ruta: [20, 13, 17, 2, 20]
  Distancia total: 37.05 km (Máxima permitida: 791 km)
  Tiemp